In [1]:
import numpy as np
import torch
from tqdm import tqdm

from scripts.das import das
from scripts.tof import time_of_flight
from utils.data import *
from utils.losses import (
    phase_error,
    total_variation,
)

torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
import tinycudann as tcnn
import commentjson as json
from torch.utils import data

def to_cuda(tensor):
    if torch.cuda.is_available():
        return tensor.cuda()
    return tensor

loss_name = "pe"
sample = "inclusion"

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get IQ data, time zeros, sampling and demodulation frequency, and element positions
iqdata, t0, fs, fd, elpos, _, _ = load_dataset(sample)
xe, _, ze = np.array(elpos)
iqdata = torch.tensor(iqdata, device=device)
xe = torch.tensor(xe, device=device)
ze = torch.tensor(ze, device=device)
t0 = torch.tensor(t0, device=device)
wl0 = ASSUMED_C / fd  # wavelength (λ)

# Sound speed grid dimensions
xc = torch.linspace(SOUND_SPEED_X_MIN, SOUND_SPEED_X_MAX, SOUND_SPEED_NXC)
zc = torch.linspace(SOUND_SPEED_Z_MIN, SOUND_SPEED_Z_MAX, SOUND_SPEED_NZC)
dxc, dzc = to_cuda(xc[1] - xc[0]), to_cuda(zc[1] - zc[0])
xc, zc = to_cuda(xc), to_cuda(zc)

# Kernels to use for loss calculations (2λ x 2λ patches)
xk, zk = torch.meshgrid((torch.arange(NXK) - (NXK - 1) / 2) * wl0 / 2,
                        (torch.arange(NZK) - (NZK - 1) / 2) * wl0 / 2, indexing="ij")
xk, zk = to_cuda(xk), to_cuda(zk)

# Kernel patch centers, distributed throughout the field of view
xpc, zpc = torch.meshgrid(torch.linspace(PHASE_ERROR_X_MIN, PHASE_ERROR_X_MAX, NXP),
                          torch.linspace(PHASE_ERROR_Z_MIN, PHASE_ERROR_Z_MAX, NZP), indexing="ij")
xpc, zpc = to_cuda(xpc), to_cuda(zpc)

# Explicit broadcasting. Dimensions will be [elements, pixels, patches]
xe = torch.reshape(xe, (-1, 1, 1))
ze = torch.reshape(ze, (-1, 1, 1))
xp = torch.reshape(xpc, (1, -1, 1)) + torch.reshape(xk, (1, 1, -1))
zp = torch.reshape(zpc, (1, -1, 1)) + torch.reshape(zk, (1, 1, -1))
xp = xp + 0 * zp  # Manual broadcasting
zp = zp + 0 * xp  # Manual broadcasting
xe, ze = to_cuda(xe), to_cuda(ze)
xp, zp = to_cuda(xp), to_cuda(zp)


# Compute time-of-flight for each {image, patch} pixel to each element
def tof_patch(model):
    return time_of_flight(xe, ze, xp, zp, model, fnum=0.5, npts=64, Dmin=3e-3)

# Define loss functions
def loss_wrapper(func, model):
    t = tof_patch(model)
    return func(iqdata, t - t0, t, fs, fd)

def sb_lc_cf_loss(func, model):
    return 1 - loss_wrapper(func, model)

def pe_loss(model):
    t = tof_patch(model)
    dphi = phase_error(iqdata, t - t0, t, fs, fd)
    valid = dphi != 0
    dphi = torch.where(valid, dphi, torch.nan)
    return torch.nanmean(torch.log1p(torch.square(100 * dphi)))

tv = lambda c: total_variation(c) * dxc * dzc

def loss(c, model):
    if loss_name == "sb":  # Speckle brightness
        return sb_lc_cf_loss("speckle_brightness", model) + tv(c) * LAMBDA_TV
    elif loss_name == "lc":  # Lag one coherence
        return torch.mean(sb_lc_cf_loss("lag_one_coherence", model)) + tv(c) * LAMBDA_TV
    elif loss_name == "cf":  # Coherence factor
        return torch.mean(sb_lc_cf_loss("coherence_factor", model)) + tv(c) * LAMBDA_TV
    elif loss_name == "pe":  # Phase error
        return pe_loss(model) + tv(c) * LAMBDA_TV
    else:
        assert False

In [3]:
# Initial survey of losses vs. global sound speed
c = to_cuda(ASSUMED_C * torch.ones((SOUND_SPEED_NXC, SOUND_SPEED_NZC)))


# Create the optimizer
with open("../utils/config.json", "r") as f:
    config = json.load(f)

l1_loss = torch.nn.L1Loss()
model = tcnn.NetworkWithInputEncoding(n_input_dims=2, n_output_dims=1,
                                      encoding_config=config["encoding"],
                                      network_config=config["network"]).to(device=device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, amsgrad=True)

xcm, ycm = torch.meshgrid(xc, zc, indexing="ij")

# Custom Dataset for coordinates
class CoordDataset(data.Dataset):
    def __init__(self, coords):
        self.coords = coords

    def __len__(self):
        return self.coords.shape[0]

    def __getitem__(self, idx):
        return self.coords[idx]

coords = torch.stack([xcm, ycm], dim=2).reshape(-1, 2)
dataset = CoordDataset(coords)
dataloader = data.DataLoader(dataset, batch_size=256, shuffle=True)

scheduler1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)

# Optimization loop
with tqdm(range(200), desc="Initializing Sound Speed", unit="iter") as pbar:
    for _ in pbar:
        model.train()
        total_loss = 0
        for batch_coords in dataloader:
            batch_coords = batch_coords.to(device=device)
            c_pre = model(batch_coords).reshape(-1, 1)
            loss_value1 = l1_loss(c_pre, c.reshape(-1, 1)[:c_pre.shape[0]])
            total_loss += loss_value1.item()
            optimizer.zero_grad()
            loss_value1.backward()
            optimizer.step()
        scheduler1.step()
        pbar.set_postfix(loss=total_loss / len(dataloader), lr=optimizer.param_groups[0]["lr"])


for param_group in optimizer.param_groups:
    param_group['lr'] = 0.1
scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)

FileNotFoundError: [Errno 2] No such file or directory: '../utils/config.json'

In [4]:
print(model(coords))

tensor([[1539.],
        [1544.],
        [1542.],
        [1539.],
        [1542.],
        [1543.],
        [1543.],
        [1543.],
        [1541.],
        [1542.],
        [1542.],
        [1540.],
        [1542.],
        [1543.],
        [1543.],
        [1541.],
        [1543.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1543.],
        [1542.],
        [1542.],
        [1543.],
        [1539.],
        [1539.],
        [1539.],
        [1542.],
        [1540.],
        [1542.],
        [1541.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1543.],
        [1543.],
        [1544.],
        [1542.],
        [1542.],
        [1539.],
        [1542.],
        [1542.],
        [1542.],
        [1541.],
        [1546.],
        [1543.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1542.],
        [1541.

In [5]:
for param in model.parameters():
    print(param.grad)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


In [6]:
for param_group in optimizer.param_groups:
    param_group['lr'] = 0.001
scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)

In [7]:
with tqdm(range(10), desc="Optimization Loop", unit="iter") as pbar:
    for _ in pbar:
        model.train()
        c_pre = model(coords).reshape(SOUND_SPEED_NXC, SOUND_SPEED_NZC)
        loss_value2 = loss(c_pre, model)
        pbar.set_postfix(loss=loss_value2.item(), lr=optimizer.param_groups[0]["lr"])
        optimizer.zero_grad()
        loss_value2.backward()
        optimizer.step()
        scheduler2.step()

Optimization Loop:   0%|          | 0/10 [00:00<?, ?iter/s]


RuntimeError: C:/Users/Auriga/AppData/Local/Temp/pip-req-build-pblpjs5j/include\tiny-cuda-nn/gpu_memory.h:591 cuMemMap(m_base_address + m_size, n_bytes_to_allocate, 0, m_handles.back(), 0) failed: CUDA_ERROR_INVALID_VALUE

In [76]:
print(model(coords))

tensor([[1539.],
        [1537.],
        [1536.],
        [1532.],
        [1530.],
        [1526.],
        [1527.],
        [1529.],
        [1522.],
        [1531.],
        [1534.],
        [1534.],
        [1529.],
        [1529.],
        [1540.],
        [1539.],
        [1534.],
        [1534.],
        [1537.],
        [1535.],
        [1529.],
        [1529.],
        [1532.],
        [1538.],
        [1539.],
        [1540.],
        [1534.],
        [1534.],
        [1531.],
        [1531.],
        [1534.],
        [1532.],
        [1538.],
        [1534.],
        [1531.],
        [1526.],
        [1529.],
        [1527.],
        [1530.],
        [1535.],
        [1532.],
        [1535.],
        [1534.],
        [1535.],
        [1532.],
        [1531.],
        [1533.],
        [1534.],
        [1535.],
        [1534.],
        [1534.],
        [1531.],
        [1539.],
        [1538.],
        [1534.],
        [1539.],
        [1534.],
        [1527.],
        [1529.

In [31]:
for param in model.parameters():
    print(param.grad)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
